# RP2040 UF2 Upload Walkthrough
This notebook mirrors the foolproof process documented in the project README and lets you rerun the exact shell commands cell by cell.

## Prerequisites
- PlatformIO CLI is installed at `~/.platformio/penv/bin/pio` (VS Code installs this automatically).
- You opened the workspace folder: `/home/felg/github/rp2040-zero-docs-pinout-platformio-project/platformio-hello-world`.
- The RP2040 is connected over USB and, if necessary, already in BOOTSEL mode (hold BOOTSEL while plugging in).

## Step 0. Prepare a dedicated Python environment
Set up a reusable virtual environment for notebook helpers, then publish its kernel so every Jupyter front end can select it without reconfiguring paths. Run the next two cells once on a new machine. If you see an "externally managed" pip error, it means you installed outside the venv—rerun these cells instead of using the system-wide `pip`.

In [ ]:
%%bash
set -euo pipefail
VENV="${HOME}/.venvs/rp2040-upload"
mkdir -p "$(dirname "${VENV}")"
python3 -m venv --upgrade "${VENV}"
"${VENV}/bin/python" -m pip install --upgrade pip ipykernel

In [ ]:
%%bash
set -euo pipefail
VENV="${HOME}/.venvs/rp2040-upload"
sudo "${VENV}/bin/python" -m ipykernel install --name rp2040-upload --display-name "Python (rp2040-upload)" --prefix /usr/local

## Step 1. Confirm the UF2 drive is mounted
Run this cell to verify that the RP2040 exposes the `RPI-RP2` mass-storage volume. If you do not see it, unplug, hold BOOTSEL, and reconnect.

In [5]:
%%bash
ls /run/media/$USER

## Step 2. Build the UF2 firmware
Invoke PlatformIO for the `pico_uf2` environment. This regenerates `.pio/build/pico_uf2/firmware.uf2` using the Arduino framework sketch.

In [ ]:
%%bash
cd /home/felg/github/rp2040-zero-docs-pinout-platformio-project/platformio-hello-world
~/.platformio/penv/bin/pio run -e pico_uf2

## Step 3. Copy the UF2 onto the RP2040
Copy the freshly built UF2 file to the mounted `RPI-RP2` volume, then flush write buffers with `sync` to ensure the transfer completes.

In [6]:
%%bash
cd /home/felg/github/rp2040-zero-docs-pinout-platformio-project/platformio-hello-world
cp .pio/build/pico_uf2/firmware.uf2 /run/media/$USER/RPI-RP2/
sync

bash: line 1: cd: /home/felg/github/rp2040-zero-docs-pinout-platformio-project/platformio-hello-world: No such file or directory
cp: cannot create regular file '/run/media/felg/RPI-RP2/': Not a directory


## Step 4. Verify the drive auto-ejected
After a successful copy, the RP2040 boots the new firmware and the `RPI-RP2` volume disappears. Rerun the directory listing to confirm.

In [ ]:
%%bash
ls /run/media/$USER

## Step 5. Optional: Reopen the serial monitor
Once the board reboots into your sketch, you can reconnect the serial monitor. Stop the monitor with `Ctrl+C` when finished.

In [ ]:
%%bash
cd /home/felg/github/rp2040-zero-docs-pinout-platformio-project/platformio-hello-world
~/.platformio/penv/bin/pio device monitor -b 115200

## Next steps
- Return to the README section "Foolproof Upload Recipes" for troubleshooting tips.
- Adapt the commands for other boards or environments by swapping the PlatformIO environment name or upload path as needed.